# Solution Overview

### Data Preprocessing / Feature Engineering

In general, our approach does not involve any additional feature engineering steps. However, to fit the model requirements and prevent model overfitting we resort to a few traditional data preprocessing and augmentation techniques by applying image resizing, normalization, blur, horizontal flips etc. 

Training dataset has been constructed by slicing original seismic volumes along one of the dimensions. Therefore, the model is designed to be trained on 2-D data. After inference (for test/holdout data), the 2-D predictions are stucked up back into 3-D volumes. 

*CAVEAT: such approach is not optimal, as seismic layers are 3 dimensional objects, so the 2-D model won't be able to decently segment 3-D objects. This approach serves as a sample pipeline only.

### Model description

As the main model, we use [UNET architecture](https://arxiv.org/abs/1505.04597) with 'resnet34' encoder pretrained on ['imagenet' dataset](https://www.image-net.org/about.php)  as such configuration proved to be a good starting point for further model training on domain specific data. 

![UNET model schema](images/UNET.png "UNET with 'resnet34' encoder")

The image is taken from [here](https://www.researchgate.net/publication/350858002_Deeply_Supervised_UNet_for_Semantic_Segmentation_to_Assist_Dermatopathological_Assessment_of_Basal_Cell_Carcinoma).


To automate and manage the model pipeline conveniently we use [Pytorch Lighning](https://lightning.ai/docs/pytorch/stable/) framework. 


With the current pipeline settings, the training process took about 10 hours (with 1 GPU (24 GB) available).

# Solution Reproduction Steps

## 1. Pipeline Configuration

Before running the pipeline, please navigate to the ```configs/config.yaml``` file and specify the actual data paths to let the pipeline know about data location. No other changes in the config file are required. 

CONFIGURATION CAVEAT:

* If the evaluation instance has more than 1 GPU, feel free to adjust the ```gpus``` parameter file to speed up the training process;

* If you encounter GPU memory problems, feel free to decrease ```batch_size``` parameter;


## 2. Environment Setup
Please, run the following command to install all needed libraries and packages.

In [ ]:
! pip install -r requirements.txt --quiet

## 3. Training step

Download the model weights from [Hugging Face](https://huggingface.co/thinkonward/challenges/tree/final-submission) before you get started. After you have downloaded them and put the `./checkpoints` directory in the root directory (`sample_final_submission`) proceed with the following instructions.

You can skip this step if you want to start with the pretrained checkpoints provided in ```./checkpoints/best```.
Otherwise, uncomment and run the following command to trigger the model training script. 

The checkpoints will be saved to ```./checkpoints``` directory.

In [ ]:
! python src/train.py

## 4. Inference step
To inference the model and form a predictions for holdout dataset please follow the instructions below. 

*INFERENCE CONFIGURATION CAVEAT:  
* Feel free to manage the input/output directories used for inference through ```inference_input_data_dir``` and ```inference_output_data_dir``` parameters in ```configs/config.yaml```;
* In order to change model checkpoints used for inference adjust the ```CHECKPOINT_WEIGHTS``` variable below.

In [5]:
import yaml
import torch
import os
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from glob import glob
from scipy import ndimage
from tqdm import tqdm

from src.train import SegmentationModule

In [22]:
# import config
config_path = "configs/config.yaml"
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
# init model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SegmentationModule(config, mode="val")

In [24]:
# path to model checkpoint
CHECKPOINT_WEIGHTS = "checkpoints/best/last.ckpt"

In [25]:
# load model weights
state_dict = torch.load(CHECKPOINT_WEIGHTS)["state_dict"]
fixed_state_dict = {
    key.replace("model.", ""): value for key, value in state_dict.items()
}

model.load_state_dict(fixed_state_dict)
model = model.cuda().eval()

In [26]:
inference_data_dir = config.get("data_paths")["inference_input_data_dir"]
# get paths to all inference volumes
inf_cube_paths = glob(os.path.join(inference_data_dir, "test_vol_*.npy"))
inf_cube_paths.sort()

In [30]:
# set of transformations applied to the inference data
inference_transform = A.Compose(
    [
        A.Lambda(image=lambda img, **kwargs: img.astype(np.float32) / 255.0),
        A.Resize(128, 320),
        A.Normalize(mean=(0.485,), std=(0.229,), max_pixel_value=1.0),
        ToTensorV2(),
    ]
)

In [31]:
inference_output_dir = config.get("data_paths")["inference_output_data_dir"]
target_shape = (300, 300, 100)

# for each inference volume
for cube_path in tqdm(inf_cube_paths):
    pred_cube = np.zeros(target_shape)
    vol = np.load(cube_path, allow_pickle=True, mmap_mode="r")
    n_slices = target_shape[0]
    # for each slice in an inference volume
    for idx in range(n_slices):
        slice = vol[idx]
        # preprocess
        transformed = inference_transform(image=slice.T)
        image = transformed["image"]
        image = torch.unsqueeze(image, 0)
        image = image.cuda()
        # get predictions
        pred_image = model(image)
        pred_image = pred_image.log_softmax(dim=1).exp()
        # post process
        pred_image = pred_image.squeeze(0).detach().cpu().numpy()
        pr = np.array(pred_image, dtype="float32")
        new_image = np.argmax(pr, axis=0)  # shape 128x320
        new_image = new_image.T  # shape 320 x 128

        new_height = target_shape[1]
        new_width = target_shape[2]
        resized_image = ndimage.zoom(
            new_image,
            (new_height / new_image.shape[0], new_width / new_image.shape[1]),
            order=1,
        )
        # assign predicted slice to prediction volume
        pred_cube[idx] = resized_image

    # save prediction volume
    cube_basename = os.path.basename(cube_path)
    pred_basename = cube_basename.replace("test", "sub")
    pred_path = os.path.join(inference_output_dir, pred_basename)
    np.save(pred_path, pred_cube, allow_pickle=True)
    print(f"{pred_path} has been created")

 33%|███▎      | 1/3 [00:10<00:21, 10.58s/it]

data/predictions/sub_vol_1.npy has been created


 67%|██████▋   | 2/3 [00:21<00:10, 10.62s/it]

data/predictions/sub_vol_2.npy has been created


100%|██████████| 3/3 [00:31<00:00, 10.54s/it]

data/predictions/sub_vol_3.npy has been created
